**RETRIEVING FULL ADDRESS WITH ONEMAP API**

In [1]:
import pandas as pd 

hdb_property = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

In [2]:
hdb_property.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [4]:
hdb_property['address'] = hdb_property['block'] + ' ' + hdb_property['street_name']

In [16]:
hdb_property.to_csv('hdb_resale_prices.csv')

In [6]:
import requests 

In [18]:
# For addresses with no postal code
def get_postal_codes(df):
    codes = []
    addresses = df['address']
    num = 0

    for x in addresses:
        search_string = str(x)
        onemap_url = "https://developers.onemap.sg/commonapi/search"
        search_params = {
            "searchVal": search_string,
            "returnGeom": "Y",
            "getAddrDetails": "Y",
            "pageNum": 1,
        }

        res = requests.get(onemap_url, params=search_params, timeout=10000)
        print(f"Request returned status code {res.status_code}")
        res = res.json()
        if res['found'] == 1:
            # should append a list so that each list item represents a singular postal code
            codes.append([search_string,
                        res['results'][0]['BLK_NO'],
                        res['results'][0]['ROAD_NAME'],
                        res['results'][0]['BUILDING'],
                        res['results'][0]['POSTAL'],
                        res['results'][0]['LATITUDE'],
                        res['results'][0]['LONGITUDE']])
                    
        elif res['found'] > 1:
            codes.append([search_string,
                        res['results'][0]['BLK_NO'],
                        res['results'][0]['ROAD_NAME'],
                        'UNKNOWN',
                        res['results'][0]['POSTAL'],
                        res['results'][0]['LATITUDE'],
                        res['results'][0]['LONGITUDE']])

        else: 
            codes.append([search_string, 'NotFound', 'NotFound', 'NotFound', 'NotFound'])
        
        num += 1
        print(num)


    df = pd.DataFrame(codes, columns=["ADDRESS", "BLK_NO", "STREET_NAME", "BUILDING_NAME", "POSTAL_CODE", "LATITUDE", "LONGITUDE"])

    return df  


In [19]:
address = get_postal_codes(hdb_property)

Request returned status code 200
1
Request returned status code 200
2
Request returned status code 200
3
Request returned status code 200
4
Request returned status code 200
5
Request returned status code 200
6
Request returned status code 200
7
Request returned status code 200
8
Request returned status code 200
9
Request returned status code 200
10
Request returned status code 200
11
Request returned status code 200
12
Request returned status code 200
13
Request returned status code 200
14
Request returned status code 200
15
Request returned status code 200
16
Request returned status code 200
17
Request returned status code 200
18
Request returned status code 200
19
Request returned status code 200
20
Request returned status code 200
21
Request returned status code 200
22
Request returned status code 200
23
Request returned status code 200
24
Request returned status code 200
25
Request returned status code 200
26
Request returned status code 200
27
Request returned status code 200
28
R

ConnectionError: HTTPSConnectionPool(host='developers.onemap.sg', port=443): Max retries exceeded with url: /commonapi/search?searchVal=4+SAGO+LANE&returnGeom=Y&getAddrDetails=Y&pageNum=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff55139e970>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
final_df = pd.merge(hdb_property, address, left_on='address', right_on='ADDRESS')

In [ ]:
final_df.to_csv('addresses_w_prices.csv')

**MERGING HDB PRICES AND HDB ADDRESS**

In [31]:
import pandas as pd

prices = pd.read_csv("hdb_resale_prices.csv")
hdb_addresses = pd.read_csv("../addresses_full.csv")

In [32]:
prices["STANDARDISED"] = prices.apply(
    lambda x: f'BLK {x["block"]} {x["street_name"].strip()}', axis=1
)

hdb_addresses["STANDARDISED"] = hdb_addresses.apply(
    lambda x: f'BLK {x["blk_no"]} {x["street"]}', axis=1
)

In [33]:
df_combined = pd.merge(prices, hdb_addresses, how="left", on="STANDARDISED")

In [34]:
from shapely.geometry import Point
# Creating a point geometry column
df_combined['geometry'] = df_combined.apply(lambda x: Point((x.LONGITUDE, x.LATITUDE)),
                                       axis = 1)

In [35]:
df_combined.columns = df_combined.columns.str.upper()

In [37]:
df_final = df_combined.drop(['UNNAMED: 0_X', 'UNNAMED: 0_Y', 'ADDRESS_X', 'ADDRESS_Y', 'BLK_NO', 'STREET'], axis = 1)

In [38]:
df_final.columns

Index(['MONTH', 'TOWN', 'FLAT_TYPE', 'BLOCK', 'STREET_NAME', 'STOREY_RANGE',
       'FLOOR_AREA_SQM', 'FLAT_MODEL', 'LEASE_COMMENCE_DATE',
       'REMAINING_LEASE', 'RESALE_PRICE', 'STANDARDISED', 'MAX_FLOOR_LVL',
       'YEAR_COMPLETED', 'RESIDENTIAL', 'COMMERCIAL', 'MARKET_HAWKER',
       'MISCELLANEOUS', 'MULTISTOREY_CARPARK', 'PRECINCT_PAVILION',
       'BLDG_CONTRACT_TOWN', 'TOTAL_DWELLING_UNITS', '1ROOM_SOLD',
       '2ROOM_SOLD', '3ROOM_SOLD', '4ROOM_SOLD', '5ROOM_SOLD', 'EXEC_SOLD',
       'MULTIGEN_SOLD', 'STUDIO_APARTMENT_SOLD', '1ROOM_RENTAL',
       '2ROOM_RENTAL', '3ROOM_RENTAL', 'OTHER_ROOM_RENTAL', 'ADDRESS',
       'STREET_NAME', 'BUILDING_NAME', 'POSTAL_CODE', 'LATITUDE', 'LONGITUDE',
       'GEOMETRY'],
      dtype='object')

In [39]:
df_final.to_csv('full_hdb_geometry_with_prices.csv', index=False)

In [40]:
check = pd.read_csv('full_hdb_geometry_with_prices.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [54]:
important_cols = ['ADDRESS', 'BLOCK', 'STREET_NAME', 'BUILDING_NAME', 'POSTAL_CODE', 'RESALE_PRICE', 'LATITUDE', 'LONGITUDE', 'GEOMETRY']
df_subset = check[important_cols]

df_subset.to_csv('hdb_with_prices.csv', index=False)

**COMBINING HDB WITH PLANNING AREA**

In [55]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [56]:
geo_df = gpd.read_file('../MP19_SZ_No_Sea/URA_MP19_SUBZONE_NO_SEA_PL.shp')

In [57]:
geo_df

,SUBZONE_NO,SUBZONE_N,SUBZONE_C,CA_IND,PLN_AREA_N,PLN_AREA_C,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D,geometry
0,1,MARINA EAST,MESZ01,Y,MARINA EAST,ME,CENTRAL REGION,CR,4FB7E5B1B9455DE0,2019-12-23,"POLYGON ((33222.981 29588.127, 33222.515 29587..."
1,5,INSTITUTION HILL,RVSZ05,Y,RIVER VALLEY,RV,CENTRAL REGION,CR,C3C22D1EE31757BD,2019-12-23,"POLYGON ((28481.446 30886.220, 28483.405 30886..."
2,1,ROBERTSON QUAY,SRSZ01,Y,SINGAPORE RIVER,SR,CENTRAL REGION,CR,87306ABAF4B67E2E,2019-12-23,"POLYGON ((28087.344 30540.999, 28087.540 30540..."
3,1,JURONG ISLAND AND BUKOM,WISZ01,N,WESTERN ISLANDS,WI,WEST REGION,WR,C87E378D3456FC35,2019-12-23,"MULTIPOLYGON (((14557.697 30447.212, 14562.889..."
4,2,FORT CANNING,MUSZ02,Y,MUSEUM,MU,CENTRAL REGION,CR,8E8F2616FFA9E019,2019-12-23,"POLYGON ((29542.526 31041.199, 29553.718 31034..."
...,...,...,...,...,...,...,...,...,...,...,...
327,1,UPPER THOMSON,BSSZ01,N,BISHAN,BS,CENTRAL REGION,CR,716DA27F6666AB0C,2019-12-23,"POLYGON ((29036.498 38365.086, 29015.440 38293..."
328,5,SHANGRI-LA,AMSZ05,N,ANG MO KIO,AM,NORTH-EAST REGION,NER,9FA8567B39D8D9D7,2019-12-23,"POLYGON ((28228.195 39216.137, 28271.551 39216..."
329,4,TOWNSVILLE,AMSZ04,N,ANG MO KIO,AM,NORTH-EAST REGION,NER,F5558D84EBC2AAA7,2019-12-23,"POLYGON ((29649.875 38978.996, 29671.324 38978..."
330,2,MARYMOUNT,BSSZ02,N,BISHAN,BS,CENTRAL REGION,CR,154AA8659ADDE6D7,2019-12-23,"POLYGON ((29469.703 36372.102, 29466.131 36348..."


In [59]:
df_subset['GEOMETRY'] = df_subset.apply(lambda x: Point((x.LONGITUDE, x.LATITUDE)),
                                       axis = 1)

<ipython-input-59-8e0bc64e1258>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['GEOMETRY'] = df_subset.apply(lambda x: Point((x.LONGITUDE, x.LATITUDE)),


In [60]:
# Create a GeoDataFrame from the original hdb_prices DataFrame
hdb_prices_geo = gpd.GeoDataFrame(df_subset, 
                                 crs = "epsg:4326",
                                 geometry = df_subset.GEOMETRY)

In [61]:
# Reprojecting CRS 
geo_df_reproj = geo_df.to_crs(3857)            
hdb_prices_geo_reproj = hdb_prices_geo.to_crs(3857)

In [62]:
hdb_prices_geo_reproj

,ADDRESS,BLOCK,STREET_NAME,BUILDING_NAME,POSTAL_CODE,RESALE_PRICE,LATITUDE,LONGITUDE,GEOMETRY,geometry
0,406 ANG MO KIO AVE 10 NIL,406,ANG MO KIO AVE 10,NIL,560406,232000.0,1.362005,103.853880,POINT (103.8538799 1.362004539),POINT (11560961.027 151631.933)
1,108 ANG MO KIO AVE 4 UNKNOWN,108,ANG MO KIO AVE 4,UNKNOWN,560108,250000.0,1.370966,103.838202,POINT (103.8382019 1.370966352),POINT (11559215.760 152629.841)
2,602 ANG MO KIO AVE 5 YIO CHU KANG GREEN,602,ANG MO KIO AVE 5,YIO CHU KANG GREEN,560602,262000.0,1.380709,103.835368,POINT (103.8353682 1.38070883),POINT (11558900.314 153714.682)
3,465 ANG MO KIO AVE 10 TECK GHEE HORIZON,465,ANG MO KIO AVE 10,TECK GHEE HORIZON,560465,265000.0,1.366201,103.857201,POINT (103.857201 1.366201041),POINT (11561330.731 152099.218)
4,601 ANG MO KIO AVE 5 UNKNOWN,601,ANG MO KIO AVE 5,UNKNOWN,560601,265000.0,1.381041,103.835132,POINT (103.8351317 1.381041348),POINT (11558873.987 153751.708)
...,...,...,...,...,...,...,...,...,...,...
135328,713 YISHUN ST 71 CHONG PANG GREEN,713,YISHUN ST 71,CHONG PANG GREEN,760713,532000.0,1.427528,103.827417,POINT (103.8274166 1.427528119),POINT (11558015.146 158928.147)
135329,391 YISHUN AVE 6 NIL,391,YISHUN AVE 6,NIL,760391,740000.0,1.429468,103.849171,POINT (103.8491714 1.429467692),POINT (11560436.880 159144.126)
135330,361 YISHUN RING RD YISHUN GLORY,361,YISHUN RING RD,YISHUN GLORY,760361,735000.0,1.428325,103.845908,POINT (103.8459083 1.428324759),POINT (11560073.633 159016.856)
135331,614 YISHUN ST 61 NIL,614,YISHUN ST 61,NIL,760614,798000.0,1.419829,103.836033,POINT (103.8360325 1.419829067),POINT (11558974.264 158070.828)


In [63]:
geo_df_reproj

,SUBZONE_NO,SUBZONE_N,SUBZONE_C,CA_IND,PLN_AREA_N,PLN_AREA_C,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D,geometry
0,1,MARINA EAST,MESZ01,Y,MARINA EAST,ME,CENTRAL REGION,CR,4FB7E5B1B9455DE0,2019-12-23,"POLYGON ((11563896.435 142930.445, 11563895.96..."
1,5,INSTITUTION HILL,RVSZ05,Y,RIVER VALLEY,RV,CENTRAL REGION,CR,C3C22D1EE31757BD,2019-12-23,"POLYGON ((11559153.720 144237.660, 11559155.68..."
2,1,ROBERTSON QUAY,SRSZ01,Y,SINGAPORE RIVER,SR,CENTRAL REGION,CR,87306ABAF4B67E2E,2019-12-23,"POLYGON ((11558759.518 143890.025, 11558759.71..."
3,1,JURONG ISLAND AND BUKOM,WISZ01,N,WESTERN ISLANDS,WI,WEST REGION,WR,C87E378D3456FC35,2019-12-23,"MULTIPOLYGON (((11545226.465 143795.261, 11545..."
4,2,FORT CANNING,MUSZ02,Y,MUSEUM,MU,CENTRAL REGION,CR,8E8F2616FFA9E019,2019-12-23,"POLYGON ((11560215.071 144393.718, 11560226.26..."
...,...,...,...,...,...,...,...,...,...,...,...
327,1,UPPER THOMSON,BSSZ01,N,BISHAN,BS,CENTRAL REGION,CR,716DA27F6666AB0C,2019-12-23,"POLYGON ((11559708.941 151768.914, 11559687.87..."
328,5,SHANGRI-LA,AMSZ05,N,ANG MO KIO,AM,NORTH-EAST REGION,NER,9FA8567B39D8D9D7,2019-12-23,"POLYGON ((11558900.412 152625.941, 11558943.78..."
329,4,TOWNSVILLE,AMSZ04,N,ANG MO KIO,AM,NORTH-EAST REGION,NER,F5558D84EBC2AAA7,2019-12-23,"POLYGON ((11560322.494 152387.131, 11560343.95..."
330,2,MARYMOUNT,BSSZ02,N,BISHAN,BS,CENTRAL REGION,CR,154AA8659ADDE6D7,2019-12-23,"POLYGON ((11560142.257 149761.947, 11560138.68..."


In [64]:
joined_gdf = gpd.sjoin(hdb_prices_geo_reproj, 
                       geo_df_reproj, 
                       how = 'left')

In [66]:
joined_gdf.columns

Index(['ADDRESS', 'BLOCK', 'STREET_NAME', 'BUILDING_NAME', 'POSTAL_CODE',
       'RESALE_PRICE', 'LATITUDE', 'LONGITUDE', 'GEOMETRY', 'geometry',
       'index_right', 'SUBZONE_NO', 'SUBZONE_N', 'SUBZONE_C', 'CA_IND',
       'PLN_AREA_N', 'PLN_AREA_C', 'REGION_N', 'REGION_C', 'INC_CRC',
       'FMEL_UPD_D'],
      dtype='object')

In [67]:
gdf_subset = joined_gdf.drop(columns='GEOMETRY')

In [68]:
gdf_subset.head()

,ADDRESS,BLOCK,STREET_NAME,BUILDING_NAME,POSTAL_CODE,RESALE_PRICE,LATITUDE,LONGITUDE,geometry,index_right,SUBZONE_NO,SUBZONE_N,SUBZONE_C,CA_IND,PLN_AREA_N,PLN_AREA_C,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D
0,406 ANG MO KIO AVE 10 NIL,406,ANG MO KIO AVE 10,NIL,560406,232000.0,1.362005,103.853880,POINT (11560961.027 151631.933),162,3,CHONG BOON,AMSZ03,N,ANG MO KIO,AM,NORTH-EAST REGION,NER,94B98B182E1B853F,2019-12-23
1,108 ANG MO KIO AVE 4 UNKNOWN,108,ANG MO KIO AVE 4,UNKNOWN,560108,250000.0,1.370966,103.838202,POINT (11559215.760 152629.841),171,6,KEBUN BAHRU,AMSZ06,N,ANG MO KIO,AM,NORTH-EAST REGION,NER,60BFB1E98CF343F9,2019-12-23
2,602 ANG MO KIO AVE 5 YIO CHU KANG GREEN,602,ANG MO KIO AVE 5,YIO CHU KANG GREEN,560602,262000.0,1.380709,103.835368,POINT (11558900.314 153714.682),241,9,YIO CHU KANG WEST,AMSZ09,N,ANG MO KIO,AM,NORTH-EAST REGION,NER,34215498CFFEE6DF,2019-12-23
3,465 ANG MO KIO AVE 10 TECK GHEE HORIZON,465,ANG MO KIO AVE 10,TECK GHEE HORIZON,560465,265000.0,1.366201,103.857201,POINT (11561330.731 152099.218),162,3,CHONG BOON,AMSZ03,N,ANG MO KIO,AM,NORTH-EAST REGION,NER,94B98B182E1B853F,2019-12-23
4,601 ANG MO KIO AVE 5 UNKNOWN,601,ANG MO KIO AVE 5,UNKNOWN,560601,265000.0,1.381041,103.835132,POINT (11558873.987 153751.708),241,9,YIO CHU KANG WEST,AMSZ09,N,ANG MO KIO,AM,NORTH-EAST REGION,NER,34215498CFFEE6DF,2019-12-23


In [69]:
gdf_subset.to_file('hdb_prices_with_planning_area.shp')

<ipython-input-69-551b827722ab>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_subset.to_file('hdb_prices_with_planning_area.shp')
